In [3]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

In [4]:
spotify_genre = pd.read_csv('working_data.nosync/genre_by_spotify_id.csv')

In [5]:
spotify_genre.head()

,Unnamed: 0,ARTIST,SPOTIFY_ID,genre
0,0,Duki Y Neo Pistea,6L74kj2KFdwfdLXTqyNvDN,"argentine hip hop,trap argentino"
1,1,Otra Onda,60xECHEcZrtFAF2uvlctP2,NaN
2,2,Zona De Arranque,5CYP7Fa3o3QDXK91AtpO7M,NaN
3,3,Duzz,5QzwfZZNjZ2fmsfHth8wuT,brazilian hip hop
4,4,Inefable,2sLbq5amMnoTNJk8Cumv17,NaN


In [6]:
spotify_genre.ARTIST.nunique()

180574

In [7]:
spotify_genre.genre.nunique()

18017

In [8]:
#Replace NaN with missing
spotify_genre.genre.fillna('missing', inplace=True)

In [9]:
spotify_genre.shape

(612405, 4)

In [83]:
#Read in Warner identified genres
warner_genre = pd.read_csv('spotify_genres.nosync/genre_keys.csv', sep=None)

/Users/ksenia/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  


In [84]:
warner_genre.head()

,﻿genre,short_genre
0,electric blues,Blues
1,garage punk blues,Blues
2,modern blues,Blues
3,soul blues,Blues
4,african gospel,Christian/Gospel


In [85]:
#Convert to dictionary
genre_2_short = warner_genre.set_index('﻿genre').T.to_dict('list')

In [86]:
#Genres obtained from Spotify
pulled_genres = spotify_genre.genre.str.split(',')

In [87]:
#Match entries with shorter genres defined by Warner

cleaned_genre = []

for line in pulled_genres:
    if line[0] == 'missing':
        cleaned_genre.append(['missing'])
    else:
        new_entry = []
        for entry in line:
            if entry in genre_2_short:
                new_entry.extend(genre_2_short[entry])
            else:
                new_entry.extend(['Other'])
        cleaned_genre.append(list(set(new_entry)))
    

In [88]:
#One hot encode

from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
encoded = pd.DataFrame(mlb.fit_transform(cleaned_genre),
                          columns=mlb.classes_,
                          index=spotify_genre.index)

In [89]:
encoded.shape

(612405, 24)

In [90]:
encoded.head()

,Blues,Christian/Gospel,Classical,Country,Dance/Electro,Disco,Dutch-House,Folk,Funk,Hip-Hop,...,Other,Pop,R&B,Reggae,Rock/Alt,Rock/Alt/Pop,Sertanejo,Singer-Songwriter,Sleep,missing
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [91]:
merged = pd.concat([spotify_genre.ARTIST, encoded], axis=1)

In [94]:
merged.drop_duplicates(inplace=True)

In [95]:
merged.ARTIST.duplicated().any()

False

In [96]:
merged.shape

(180574, 25)

In [97]:
merged.to_csv('working_data.nosync/spotify_genre_by_artist.csv')